In [1]:
import sys
sys.path.append('..')
from src.models.pipeline import pipeline
import glob
import json
import os
import pandas as pd
import ast
from datetime import datetime
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

In [2]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript
#print(transcripts_jsons)

'''Read the videos metadata to perform on them the segmentation'''
df_videos = pd.read_csv(topic_dataset_path)

''' Transfer topic shifts time to seconds units instead HH:MM:SS'''

def topic_shifts_seconds(topic_shifts):
    tp_shift_sec=[]
    for tp in topic_shifts:
        intervals = tp.split(':')
        seconds = int(intervals[2])
        minutes = int(intervals[1]) * 60
        hours = int(intervals[0]) * 60 *60
        tp_shift_sec.append(seconds + minutes + hours)
    return tp_shift_sec


for video_id in transcripts_jsons.keys():    
    df_videos.at[df_videos['video id'] == video_id,'topic shifts(ends)'] =\
    topic_shifts_seconds(\
                         df_videos[df_videos['video id'] == \
                                   video_id]['topic shifts(ends)'])

In [34]:
'''Create an array of functions to optimize'''
def build_function_to_optimize(workflow_label):
    pipeline_steps_values = workflow_label.split('-')
    steps_labels = ['segment','vectorize','similarity','added_filter','clustering']
    _pipeline = {}
    for key,value in list(zip(steps_labels,pipeline_steps_values)):
        _pipeline[key]=value
    
    '''takingcare of filters'''
    filter_params = {'filter_type': None,'mask_shape': None,'sim_thresh':None,'is_min_thresh':True}
    # if we have image filter 
    if _pipeline['added_filter'] != 'None':
        vals = _pipeline['added_filter'].split('_')
        filter_params['filter_type'] = vals[0]
        filter_params['mask_shape'] = ast.literal_eval(vals[1])
        
    #print(filter_params)    
    #print(_pipeline)
    '''Cosine and spectral clustering'''
    if _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        def _f(window_size,step_size,sim_thresh,n_clusters):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                window_size=window_size,step_size=step_size,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters}
                               )

        return _f

In [35]:
param_bounds = {
    'sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering':{
        "window_size": (20,200),
        "step_size":(10,60),
        'sim_thresh':(0.2,0.9),
        'n_clusters': (10,18)
    },
    'sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering':{
        "window_size": (20,150),
        "step_size":(10,60),
        'sim_thresh':(0.3,0.6),
        'n_clusters': (12,18)
    },
    'sliding_window-tfidf-cosine-None-spectral_clustering':{
        "window_size": (20,150),
        "step_size":(10,60),
        'sim_thresh':(0.3,0.6),
        'n_clusters': (12,18)
    }
}

In [38]:
models_path  = '../models/bayesian_opt'
''' Finding optimization for each video'''

workflows = list(param_bounds.keys())

for workflow in workflows:
    print('Training workflow %s' %(workflow))
    function_to_optimized = build_function_to_optimize(workflow)
    '''Define the optimizer'''    
    optimizer = BayesianOptimization(
                f=function_to_optimized,
                pbounds=param_bounds[workflow],
                verbose=2,
                random_state=1
                )
    
    model_file_path = os.path.join(models_path,('%s.json' %(workflow)))
    if os.path.isfile(model_file_path):
        load_logs(optimizer,logs=[model_file_path])
    else:
        logger = JSONLogger(path=model_file_path)
        optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)


    for vid in transcripts_jsons.keys():
        '''Define the function to optimize'''
        video_metadata = df_videos.loc[df_videos['video id'] == vid]
        groundbase = video_metadata['topic shifts(ends)'].values.tolist()[:-1]
        transcripts = transcripts_jsons[vid]

        print("Running the algorithm on %s " %(vid))
        '''Run the algorithm'''
        optimizer.maximize(
            init_points = 1,
            n_iter = 7
        )

Training workflow sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering
Running the algorithm on 2mC1uqwEmWQ 
|   iter    |  target   | n_clus... | sim_th... | step_size | window... |
-------------------------------------------------------------------------
|  1        |  0.0      |  18.0     |  0.2      |  31.16    |  46.41    |
|  2        |  0.0      |  17.24    |  0.3782   |  59.72    |  197.9    |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  3        |  0.0      |  17.84    |  0.8651   |  35.14    |  107.5    |
|  4        |  0.3333   |  11.15    |  0.4821   |  10.13    |  45.7     |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  5        |  0.0      |  10.0     |  0.9      |  60.0     |  44.4     |


...

|  46       |  0.0      |  12.0     |  0.6      |  10.0     |  71.89    |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  47       |  0.0      |  12.0     |  0.3      |  60.0     |  68.6     |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  48       |  0.0      |  18.0     |  0.3      |  10.0     |  45.23    |


In [7]:
("%s %s" %(1,2))

'1 2'